# Entrenar un modelo de clasificación con Automated Machine Learning

Hay muchos tipos de algoritmos de aprendizaje automático que puede usar para entrenar un modelo y, a veces, no es fácil determinar el más eficaz para los requisitos de predicción y datos concretos. Además, puede afectar significativamente al rendimiento predictivo de un modelo mediante el preprocesamiento de los datos de entrenamiento, con técnicas como las de normalización, imputación de características que faltan y otras. En la búsqueda del mejor modelo para los requisitos, es posible que tenga que probar muchas combinaciones de algoritmos y transformaciones de preprocesamiento, para lo que se necesita mucho tiempo y recursos de proceso.

Azure Machine Learning permite automatizar la comparación de modelos entrenados con diferentes algoritmos y opciones de preprocesamiento. Puede usar la interfaz visual de [Azure Machine Learning Studio](https://ml/azure.com) o el SDK de Python (v2) para aprovechar esta capacidad. El SDK de Python le ofrece un mayor control sobre la configuración del trabajo automatizado de aprendizaje automático, pero la interfaz visual es más fácil de usar.

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Preparación de los datos

No es necesario crear un script de entrenamiento para el aprendizaje automático automatizado, pero sí cargar los datos de entrenamiento. 

En este caso, usará un conjunto de datos que contiene detalles de pacientes con diabetes. 

Para pasar un conjunto de datos como entrada a un trabajo de aprendizaje automático automatizado, los datos deben estar en formato tabular e incluir una columna de destino. Para que los datos se interpreten como un conjunto de datos tabular, el conjunto de datos de entrada debe ser **mlTable**.

Ya se ha creado un recurso de datos de MLTable durante la configuración. Para explorar el recurso de datos, vaya a la página **Datos**. Aquí recuperará el recurso de datos especificando su nombre `diabetes-training-table` y versión `1`. 

In [ ]:
## Configuración del trabajo automatizado de aprendizaje automático

Ya está listo para configurar el experimento de aprendizaje automático automatizado.

Al ejecutar el código siguiente, creará un trabajo de aprendizaje automático automatizado que:

- Usa el clúster de proceso denominado `aml-cluster`
- Establece `Diabetic` como columna de destino
- Establece `accuracy` como métrica principal
- Agota el tiempo de espera después de `60` minutos del tiempo total de entrenamiento 
- Entrena un máximo de `5` modelos
- No se entrenará ningún modelo con el algoritmo `LogisticRegression`

In [ ]:
## Cancelar un trabajo de aprendizaje automático automatizado

Ya tiene todo listo. Ahora se ejecutará el experimento de aprendizaje automático automatizado.

> **Nota**: Esto puede tardar algún tiempo.

Mientras se ejecuta el trabajo, puede supervisarlo en Studio.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.